## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [10]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [3]:
# answer goes here
data = data[data['GS'] >= 20]
data = data[data['MP'] >= 10]

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [4]:
# answer goes here
data = data[['PTS','TRB', 'AST', 'STL', 'BLK', 'TOV']].reset_index(drop=True)

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [5]:
# answer goes here
sc = StandardScaler()
data_sc = sc.fit_transform(data)
data_sc = pd.DataFrame(data_sc, columns=data.columns)

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [6]:
# instantiate and fit K-Means model
kmeans = KMeans(n_clusters=3)
# save predictions to new column
kmeans.fit_predict(data_sc)

# unscale and print cluster centers
clusters = sc.inverse_transform(kmeans.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=data_sc.columns)
clusters.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,10.505405,3.949775,2.063739,0.804730,0.374324,1.235135
1,13.735359,8.605525,1.914917,0.777348,1.230387,1.668508
2,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765


In [7]:
# repeat for Gaussian model
gmm3 = GaussianMixture(n_components=3, covariance_type='full')
gmm3.fit_predict(data_sc)
clusters = sc.inverse_transform(gmm3.means_)
clusters = pd.DataFrame(clusters, columns=data_sc.columns)
clusters.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,14.449177,8.481379,2.259574,0.775804,1.221910,1.830776
1,10.239585,4.476528,1.582674,0.780572,0.428468,1.105656
2,15.898592,4.178026,4.812924,1.176953,0.371760,2.246579


In [8]:
# instantiate and fit K-Means model for 7 clusters
kmeans = KMeans(n_clusters=7)
# save predictions to new column
kmeans.fit_predict(data_sc)

# unscale and print cluster centers
clusters = sc.inverse_transform(kmeans.cluster_centers_)
clusters = pd.DataFrame(clusters, columns=data_sc.columns)
clusters.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,11.492000,7.265333,1.666667,0.654667,0.937333,1.388000
1,24.603226,8.374194,7.803226,1.603226,0.809677,3.941935
2,14.282143,3.709524,3.152976,0.838095,0.276786,1.710119
3,11.285714,4.885714,2.130769,1.354945,0.498901,1.300000
4,7.517318,3.484916,1.373743,0.592179,0.355307,0.884916
5,17.755000,10.473333,2.343333,0.921667,1.743333,2.140000
6,18.525600,4.716000,5.784800,1.336800,0.400800,2.627200


In [9]:
# repeat for Gaussian model
gmm = GaussianMixture(n_components=7, covariance_type='full')
gmm.fit_predict(data_sc)
gmm_clusters = sc.inverse_transform(gmm.means_)
clusters = pd.DataFrame(clusters, columns=data_sc.columns)
clusters.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,11.492000,7.265333,1.666667,0.654667,0.937333,1.388000
1,24.603226,8.374194,7.803226,1.603226,0.809677,3.941935
2,14.282143,3.709524,3.152976,0.838095,0.276786,1.710119
3,11.285714,4.885714,2.130769,1.354945,0.498901,1.300000
4,7.517318,3.484916,1.373743,0.592179,0.355307,0.884916
5,17.755000,10.473333,2.343333,0.921667,1.743333,2.140000
6,18.525600,4.716000,5.784800,1.336800,0.400800,2.627200


Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [11]:
# get cluster probabilities for each observation
probs = pd.DataFrame(gmm3.predict_proba(data_sc))
# get cluster entropy vals from probabilities
ent = pd.DataFrame(entr(probs))

In [12]:
# create column for total entropy for each row
data['total'] = ent.sum(axis=1)
# sort df by entropy value
data.sort_values(by=['total'], inplace=True)

In [17]:
print('The 5 most confident labels (least entropy):')
data[:5]

The 5 most confident labels (least entropy):


,PTS,TRB,AST,STL,BLK,TOV,total
797,14.2,11.8,0.4,0.6,3.7,1.9,3.393115e-38
182,13.3,7.2,1.6,0.8,2.7,1.4,3.753057e-21
351,22.7,6.6,1.2,0.8,2.4,1.9,2.466502e-17
252,28.1,11.1,2.3,1.5,2.6,2.2,2.595889e-17
454,20.2,7.8,2.1,0.9,2.5,3.8,7.364098e-16


In [16]:
print('5 least confident labels (most entropy):')
data[-5:]

5 least confident labels (most entropy):


,PTS,TRB,AST,STL,BLK,TOV,total
109,7.8,3.2,2.5,0.6,0.5,1.3,0.912067
433,9.2,4.2,0.7,0.8,0.9,1.3,0.925587
343,11.5,5.7,2.7,0.8,0.5,1.9,0.986075
741,12.0,5.5,1.9,0.9,0.5,2.1,1.057616
13,11.5,4.6,2.9,0.4,0.5,1.5,1.068226
